<a href="https://colab.research.google.com/github/Ddkaba/AutoLogReg/blob/main/AutoLogReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import patoolib
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    hamming_loss, f1_score, precision_score, recall_score, classification_report
)

In [2]:
# Укажите имя вашего RAR-архива
rar_file = 'complaints.rar'  # замените на имя вашего файла

# Распакуйте архив в папку 'extracted_files'
patoolib.extract_archive(rar_file, outdir='extracted_files')

# Укажите имя CSV-файла, который вы хотите прочитать
csv_file = 'extracted_files/complaints.csv'  # замените на имя вашего CSV-файла

# Прочитайте CSV-файл
cars_df = pd.read_csv(csv_file)

# Посмотрите первые 5 строк
print(cars_df.head())

INFO patool: Extracting complaints.rar ...
INFO:patool:Extracting complaints.rar ...
INFO patool: running /usr/bin/unrar x -kb -or -- /content/complaints.rar
INFO:patool:running /usr/bin/unrar x -kb -or -- /content/complaints.rar
INFO patool: ... complaints.rar extracted to `extracted_files'.
INFO:patool:... complaints.rar extracted to `extracted_files'.


   odiNumber                manufacturer  crash   fire  numberOfInjuries  \
0   10702705           Kia America, Inc.   True  False                 0   
1   10713088           Kia America, Inc.  False  False                 0   
2   10713503    ALUMA TOWER COMPANY, INC  False   True                 0   
3   10715078           Kia America, Inc.  False  False                 0   
4   10715844  Volvo Trucks North America  False  False                 0   

   numberOfDeaths dateOfIncident dateComplaintFiled          vin  \
0               0     03/24/2015         03/30/2015  5XYPG4A36GG   
1               0     04/24/2015         04/27/2015  5XYPH4A15GG   
2               0     04/28/2015         04/29/2015  1YGAE1629GB   
3               0     04/30/2015         05/07/2015          NaN   
4               0     03/06/2015         05/12/2015  4V4NC9EH7GN   

          components                                            summary  \
0   SEAT BELTS,SEATS        SEAT BELT DID NOT UNLATCH AFTER

In [3]:
print("Общая информация")
print(cars_df.info())

print(f"Количество записей (объектов): {cars_df.shape[0]}")
print(f"Количество признаков (фич): {cars_df.shape[1]}")

print("\nНазвания столбцов:")
print(cars_df.columns.tolist())

print("\nТипы данных:")
print(cars_df.dtypes)

print("\nПропущенные значения:")
missing_values = cars_df.isnull().sum()
print(missing_values)
print(f"Общее количество пропущенных значений: {missing_values.sum()}")

print("Целевая переменная")
target_column = 'components'
if target_column in cars_df.columns:
    print(f"\nЦелевая переменная: {target_column}")
    print(f"Тип данных целевой переменной: {cars_df[target_column].dtype}")
    unique_values = cars_df[target_column].unique()
    print(f"Уникальные значения целевой переменной (первые 20): {unique_values[:20]}")
    print(f"Всего уникальных значений: {unique_values.size}")
    if cars_df[target_column].nunique() <= 20:
        print("Распределение классов:")
        print(cars_df[target_column].value_counts())
        print("Процентное соотношение классов:")
        print(cars_df[target_column].value_counts(normalize=True) * 100)

print("Статистика")
print(cars_df.describe())

print("Анализ кат. признаков")
categorical_features = []
for col in cars_df.columns:
    unique_values = cars_df[col].nunique(dropna=True)
    if unique_values <= 10:
        categorical_features.append(col)
        print(f"{col}: {unique_values} уникальных значений - {cars_df[col].unique()}")

print(f"\nВсего категориальных признаков: {len(categorical_features)}")


Общая информация
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233820 entries, 0 to 233819
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   odiNumber           233820 non-null  int64 
 1   manufacturer        233802 non-null  object
 2   crash               233820 non-null  bool  
 3   fire                233820 non-null  bool  
 4   numberOfInjuries    233820 non-null  int64 
 5   numberOfDeaths      233820 non-null  int64 
 6   dateOfIncident      233820 non-null  object
 7   dateComplaintFiled  233820 non-null  object
 8   vin                 228677 non-null  object
 9   components          233810 non-null  object
 10  summary             233809 non-null  object
 11  products            233820 non-null  object
 12  make                233820 non-null  object
 13  model               233820 non-null  object
 14  modelYear           233820 non-null  int64 
dtypes: bool(2), int64(4), object(9)
me

In [4]:
# Удаление нерелевантных признаков
cols_to_drop = ['odiNumber', 'manufacturer', 'vin', 'products', 'dateOfIncident','dateComplaintFiled']

# Выбираем рабочий датафрейм: если уже есть сгенерированный (cars_df), используем его,
# иначе — исходный cars_df.
df_ref = cars_df if 'dataset' in globals() else cars_df

print('До:', df_ref.shape)
# Удаляем только те признаки, которые реально присутствуют в датафрейме
df_ref.drop(columns=[c for c in cols_to_drop if c in df_ref.columns], axis=1, inplace=True)
print('После:', df_ref.shape)
print('Текущие столбцы:')
print(df_ref.columns.tolist())

# Если мы работаем с исходным dataset (dataset отсутствует),
# обновляем его, чтобы изменения сохранились.
if 'cars_df' not in globals():
    cars_df = df_ref

До: (233820, 15)
После: (233820, 9)
Текущие столбцы:
['crash', 'fire', 'numberOfInjuries', 'numberOfDeaths', 'components', 'summary', 'make', 'model', 'modelYear']


In [5]:
cars_df = cars_df.dropna(subset=['components'])
print(cars_df.shape)
cars_df = cars_df.copy()
cars_df['num_components'] = cars_df['components'].apply(lambda x: len(x.split(',')) if isinstance(x, str) else 0)
print(cars_df.shape)
print('Текущие столбцы:')
print(cars_df.columns.tolist())

(233810, 9)
(233810, 10)
Текущие столбцы:
['crash', 'fire', 'numberOfInjuries', 'numberOfDeaths', 'components', 'summary', 'make', 'model', 'modelYear', 'num_components']


In [6]:
def standardize_features(dataset):
    """
    Стандартизирует числовые признаки в датасете (z = (x - mean) / std).

    Параметры:
        dataset (pd.DataFrame): Входной датасет.

    Возвращает:
        pd.DataFrame: Стандартизированный датасет.
    """
    # Создаем копию, чтобы не изменять оригинал
    base_df = dataset.copy()

    # Отбираем числовые признаки
    feature_cols = base_df.select_dtypes(include=[np.number]).columns.tolist()

    # Проверка, есть ли числовые признаки
    if not feature_cols:
        raise ValueError("В датасете нет числовых признаков для стандартизации.")

    # Стандартизация
    scaler = StandardScaler()
    scaled = scaler.fit_transform(base_df[feature_cols])

    # Создаем DataFrame с стандартизированными признаками
    standardized_df = pd.DataFrame(scaled, columns=feature_cols, index=base_df.index)

    # Возвращаем все столбцы, заменяя числовые на стандартизированные
    base_df[feature_cols] = standardized_df[feature_cols]

    # Вывод информации
    print('Стандартизированы признаки:', feature_cols)
    print('Форма:', standardized_df.shape)

    # Проверка средних и стандартных отклонений
    means = base_df[feature_cols].mean().round(4)
    stdevs = base_df[feature_cols].std(ddof=0).round(4)
    print('\nСредние по столбцам (ожид. ≈ 0):')
    print(means)
    print('\nСт. отклонения (ожид. ≈ 1):')
    print(stdevs)
    print('\n')

    return base_df

# Пример использования
cars_df_standardized = standardize_features(cars_df)

print("\nТипы данных:")
print(cars_df_standardized.dtypes)


Стандартизированы признаки: ['numberOfInjuries', 'numberOfDeaths', 'modelYear', 'num_components']
Форма: (233810, 4)

Средние по столбцам (ожид. ≈ 0):
numberOfInjuries    0.0
numberOfDeaths      0.0
modelYear          -0.0
num_components     -0.0
dtype: float64

Ст. отклонения (ожид. ≈ 1):
numberOfInjuries    1.0
numberOfDeaths      1.0
modelYear           1.0
num_components      1.0
dtype: float64



Типы данных:
crash                  bool
fire                   bool
numberOfInjuries    float64
numberOfDeaths      float64
components           object
summary              object
make                 object
model                object
modelYear           float64
num_components      float64
dtype: object


In [7]:
cars_df_standardized['components_list'] = (
    cars_df_standardized['components']
    .str.split(',')
    .apply(lambda x: [i.strip() for i in x] if isinstance(x, list) else [])
)

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(cars_df_standardized['components_list'])

# Фильтруем редкие компоненты
min_support = 50
valid_mask = y.sum(axis=0) >= min_support
y = y[:, valid_mask]
component_names = mlb.classes_[valid_mask]
print(f"Оставлено компонентов: {len(component_names)}")

Оставлено компонентов: 40


In [8]:
# 2. X — без summary или с малым TF-IDF
X = cars_df_standardized[['make', 'model', 'modelYear']].copy()

# 3. Пайплайн — без dense, с малым TF-IDF
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['make', 'model']),  # sparse по умолчанию
    ('num', 'passthrough', ['modelYear']),
    # ('text', TfidfVectorizer(max_features=300, stop_words='english'), 'summary')  # раскомментируй позже
], sparse_threshold=1.0)  # сохраняем разреженность

In [9]:
model = Pipeline([
    ('prep', preprocessor),
    ('clf', MultiOutputClassifier(
        LGBMClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            n_jobs=-1,
            verbose=-1,
            random_state=42
        ),
        n_jobs=-1  # параллелим по компонентам
    ))
])

# 5. Обучение с таймером
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y.sum(axis=1)
)

model.fit(X_train, y_train)

print(f"ОБУЧЕНИЕ ЗАВЕРШЕНО")

ОБУЧЕНИЕ ЗАВЕРШЕНО


In [10]:
y_pred = model.predict(X_test)

print("=== ОСНОВНЫЕ МЕТРИКИ ===")
print(f"Hamming Loss:     {hamming_loss(y_test, y_pred):.4f} ← чем ниже, тем лучше")
print(f"F1 Micro:         {f1_score(y_test, y_pred, average='micro'):.4f}")
print(f"F1 Macro:         {f1_score(y_test, y_pred, average='macro'):.4f}")
print(f"Precision Micro:  {precision_score(y_test, y_pred, average='micro'):.4f}")
print(f"Recall Micro:     {recall_score(y_test, y_pred, average='micro'):.4f}")

=== ОСНОВНЫЕ МЕТРИКИ ===
Hamming Loss:     0.0351 ← чем ниже, тем лучше
F1 Micro:         0.0833
F1 Macro:         0.0406
Precision Micro:  0.7027
Recall Micro:     0.0443


In [14]:
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
import numpy as np

print("=== ШАГ 1: Получаем вероятности ===")
y_proba = model.predict_proba(X_test)
y_proba = np.array([prob[:, 1] for prob in y_proba]).T  # (n_samples, n_components)

print("=== ШАГ 2: Подбираем лучший порог ===")
thresholds = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
best_thresh = 0.1
best_f1 = 0

for thresh in thresholds:
    y_pred = (y_proba > thresh).astype(int)
    f1 = f1_score(y_test, y_pred, average='micro')
    print(f"  Порог {thresh:.2f} → F1 Micro: {f1:.4f}")
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = thresh

print(f"\nЛУЧШИЙ ПОРОГ: {best_thresh} → F1 Micro: {best_f1:.4f}")

# === ШАГ 3: Финальные метрики с лучшим порогом ===
y_pred_best = (y_proba > best_thresh).astype(int)

print("\n=== ФИНАЛЬНЫЕ МЕТРИКИ ===")
print(f"F1 Micro:     {f1_score(y_test, y_pred_best, average='micro'):.4f}")
print(f"Precision:    {precision_score(y_test, y_pred_best, average='micro'):.4f}")
print(f"Recall:       {recall_score(y_test, y_pred_best, average='micro'):.4f}")
print(f"Hamming Loss: {hamming_loss(y_test, y_pred_best):.4f}")

=== ШАГ 1: Получаем вероятности ===


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


=== ШАГ 2: Подбираем лучший порог ===
  Порог 0.05 → F1 Micro: 0.2546
  Порог 0.10 → F1 Micro: 0.3114
  Порог 0.15 → F1 Micro: 0.3398
  Порог 0.20 → F1 Micro: 0.3348
  Порог 0.25 → F1 Micro: 0.2928
  Порог 0.30 → F1 Micro: 0.2537

ЛУЧШИЙ ПОРОГ: 0.15 → F1 Micro: 0.3398

=== ФИНАЛЬНЫЕ МЕТРИКИ ===
F1 Micro:     0.3398
Precision:    0.2488
Recall:       0.5359
Hamming Loss: 0.0750


In [15]:
import joblib

joblib.dump(model, 'failure_predictor_v1.pkl')
joblib.dump(mlb, 'mlb_components.pkl')
joblib.dump(best_thresh, 'best_threshold.pkl')
joblib.dump(component_names, 'component_names.pkl')

print("МОДЕЛЬ СОХРАНЕНА: failure_predictor_v1.pkl")

МОДЕЛЬ СОХРАНЕНА: failure_predictor_v1.pkl


In [16]:
def predict_failure(make, model_name, model_year_z, summary=""):
    """
    Предсказывает, какие компоненты могут сломаться
    """
    # Приводим modelYear к z-score (если не знаешь — используй 0.0)
    new_car = pd.DataFrame([{
        'make': make.upper(),
        'model': model_name.upper(),
        'modelYear': model_year_z,
        'summary': summary
    }])

    proba = model.predict_proba(new_car)
    proba = np.array([p[:, 1] for p in proba]).T[0]

    predicted = component_names[proba > best_thresh]
    scores = proba[proba > best_thresh]

    result = sorted(zip(predicted, scores), key=lambda x: x[1], reverse=True)
    return result

# Пример:
print("KIA SORENTO 2016:")
for comp, score in predict_failure('KIA', 'SORENTO', 0.0):
    print(f"  • {comp:20} → {score:.3f}")

KIA SORENTO 2016:


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


  • ENGINE               → 0.439
  • UNKNOWN OR OTHER     → 0.163
  • ELECTRICAL SYSTEM    → 0.152


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
import numpy as np
import time

print("Запускаем УМНОЕ обучение с summary...")

# 1. Разделяем данные ДО пайплайна
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X[['make', 'model', 'modelYear', 'summary']], y, test_size=0.2, random_state=42
)

# 2. Пайплайн БЕЗ TF-IDF
preprocessor_no_text = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['make', 'model']),
    ('num', 'passthrough', ['modelYear'])
], sparse_threshold=1.0)

# 3. Обучаем базовые признаки
print("Шаг 1: Обработка make, model, year...")
X_train_base = preprocessor_no_text.fit_transform(X_train_raw[['make', 'model', 'modelYear']])
X_test_base = preprocessor_no_text.transform(X_test_raw[['make', 'model', 'modelYear']])

# 4. TF-IDF ОТДЕЛЬНО — с малым размером
print("Шаг 2: TF-IDF по summary...")
tfidf = TfidfVectorizer(max_features=300, stop_words='english', ngram_range=(1,1))
X_train_tfidf = tfidf.fit_transform(X_train_raw['summary'])
X_test_tfidf = tfidf.transform(X_test_raw['summary'])

# 5. Объединяем вручную
from scipy.sparse import hstack
X_train_final = hstack([X_train_base, X_train_tfidf])
X_test_final = hstack([X_test_base, X_test_tfidf])

print(f"Финальная матрица: {X_train_final.shape}")

# 6. Обучаем LightGBM — БЕЗ MultiOutput (быстрее!)
print("Шаг 3: Обучение LightGBM...")
start = time.time()

model_final = MultiOutputClassifier(
    LGBMClassifier(n_estimators=80, max_depth=5, n_jobs=1, verbose=-1),  # n_jobs=1!
    n_jobs=1  # ОТКЛЮЧИ ПАРАЛЛЕЛИЗМ!
)

model_final.fit(X_train_final, y_train)
end = time.time()

print(f"ГОТОВО ЗА: {(end-start)/60:.1f} МИНУТ")

Запускаем УМНОЕ обучение с summary...


KeyError: "['summary'] not in index"

In [26]:
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
import numpy as np

print("=== ПОДБОР ЛУЧШЕГО ПОРОГА ===")
y_proba = np.array([est.predict_proba(X_test_final)[:, 1] for est in model_final.estimators_]).T

thresholds = [0.05, 0.1, 0.15, 0.2, 0.25]
best_thresh = 0.1
best_f1 = 0

for t in thresholds:
    y_pred = (y_proba > t).astype(int)
    f1 = f1_score(y_test, y_pred, average='micro')
    print(f"  Порог {t:.2f} → F1 Micro: {f1:.4f}")
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"\nЛУЧШИЙ ПОРОГ: {best_thresh} → F1 Micro: {best_f1:.4f}")

# Финальные метрики
y_pred_best = (y_proba > best_thresh).astype(int)
print("\n=== ФИНАЛЬНЫЕ МЕТРИКИ (с summary) ===")
print(f"F1 Micro:     {f1_score(y_test, y_pred_best, average='micro'):.4f}")
print(f"Precision:    {precision_score(y_test, y_pred_best, average='micro'):.4f}")
print(f"Recall:       {recall_score(y_test, y_pred_best, average='micro'):.4f}")
print(f"Hamming Loss: {hamming_loss(y_test, y_pred_best):.4f}")

=== ПОДБОР ЛУЧШЕГО ПОРОГА ===


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


  Порог 0.05 → F1 Micro: 0.4042
  Порог 0.10 → F1 Micro: 0.5031
  Порог 0.15 → F1 Micro: 0.5588
  Порог 0.20 → F1 Micro: 0.5832
  Порог 0.25 → F1 Micro: 0.5950

ЛУЧШИЙ ПОРОГ: 0.25 → F1 Micro: 0.5950

=== ФИНАЛЬНЫЕ МЕТРИКИ (с summary) ===
F1 Micro:     0.5950
Precision:    0.5811
Recall:       0.6095
Hamming Loss: 0.0298


In [44]:
# =============================================
# 1. ЗАГРУЗКА (ПОСЛЕ ПЕРЕЗАПУСКА)
# =============================================
import joblib
import pickle
import pandas as pd
import numpy as np
from scipy.sparse import hstack

print("Загружаем модель...")
model = joblib.load('failure_predictor_final.pkl')        # ← Теперь это MultiOutputClassifier!
prep = joblib.load('preprocessor_base.pkl')
with open('model_metadata.pkl', 'rb') as f:
    meta = pickle.load(f)

THRESHOLD = meta['best_threshold']
COMPONENTS = meta['component_names']

# Категории
known_makes = prep.named_transformers_['cat'].categories_[0]
known_models = prep.named_transformers_['cat'].categories_[1]

print(f"Готово! Порог: {THRESHOLD}, Компонентов: {len(COMPONENTS)}")
print(f"Марок: {len(known_makes)}, Моделей: {len(known_models)}")

# =============================================
# 2. ФУНКЦИЯ ПРЕДСКАЗАНИЯ
# =============================================
def predict_failures(make, model, year_raw, threshold=THRESHOLD):
    make = make.upper().strip()
    model = model.upper().strip()

    if make not in known_makes:
        return [("НЕИЗВЕСТНАЯ МАРКА", 0.0)]
    if model not in known_models:
        return [("НЕИЗВЕСТНАЯ МОДЕЛЬ", 0.0)]

    try:
        year_z = (year_raw - 2015) / 3
        df = pd.DataFrame([{'make': make, 'model': model, 'modelYear': year_z}])
        X = prep.transform(df[['make', 'model', 'modelYear']])

        if X.nnz == 0:
            return [("НЕТ ДАННЫХ", 0.0)]

        proba = np.array([est.predict_proba(X)[:, 1][0] for est in model.estimators_])
        mask = proba > threshold
        if not mask.any():
            return [("РИСКИ МИНИМАЛЬНЫ", 0.0)]

        risks = (proba[mask] * 100).round(1)
        comps = COMPONENTS[mask]
        return sorted(zip(comps, risks), key=lambda x: x[1], reverse=True)[:10]

    except Exception as e:
        return [("ОШИБКА", 0.0), (str(e)[:50], 0.0)]

# =============================================
# 3. ТЕСТ НА РЕАЛЬНЫХ МАРКАХ
# =============================================
print("\n" + "="*60)
print("ПРЕДСКАЗАНИЕ ДЛЯ ПОПУЛЯРНЫХ АВТО")
print("="*60)

test_cars = [
    ('NISSAN', 'ROGUE', 2016),
    ('FORD', 'ESCAPE', 2014),
    ('TOYOTA', 'SIENNA', 2018),
    ('HONDA', 'CIVIC', 2011),
    ('CHEVROLET', 'EQUINOX', 2015),
]

for make, model, year in test_cars:
    print(f"\n{make} {model} {year}:")
    results = predict_failures(make, model, year)
    for comp, risk in results:
        print(f"  • {comp:25} → {risk:5.1f}%")

Загружаем модель...
Готово! Порог: 0.25, Компонентов: 40
Марок: 331, Моделей: 1747

ПРЕДСКАЗАНИЕ ДЛЯ ПОПУЛЯРНЫХ АВТО

NISSAN ROGUE 2016:
  • ОШИБКА                    →   0.0%
  • 'str' object has no attribute 'estimators_' →   0.0%

FORD ESCAPE 2014:
  • ОШИБКА                    →   0.0%
  • 'str' object has no attribute 'estimators_' →   0.0%

TOYOTA SIENNA 2018:
  • ОШИБКА                    →   0.0%
  • 'str' object has no attribute 'estimators_' →   0.0%

HONDA CIVIC 2011:
  • ОШИБКА                    →   0.0%
  • 'str' object has no attribute 'estimators_' →   0.0%

CHEVROLET EQUINOX 2015:
  • ОШИБКА                    →   0.0%
  • 'str' object has no attribute 'estimators_' →   0.0%
